In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import getpass
import os
import langchain
from langchain.schema.messages import HumanMessage, SystemMessage,AIMessage
from langchain_openai import AzureChatOpenAI


# 2. Create model
model = AzureChatOpenAI(
    azure_endpoint="https://12205-m2hl4tqk-eastus.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-08-01-preview",
    azure_deployment="gpt-35-turbo",
    openai_api_version="2024-08-01-preview",
    api_key="d7f27353b3b3463bb02b2708df922f35",  
)


In [21]:
parser=StrOutputParser()
chain=model|parser

### 首先直接使用模型。model是LangChain“运行接口”的实例，这意味着它们提供了一个标准接口供我们与之交互。要简单地调用模型，我们可以将消息列表传递给.invoke方法。

In [22]:
messages=[HumanMessage(content='i am ed'),SystemMessage(content='answer word<10')]


In [23]:
chain.invoke(messages)

'Hi!'

### 模型本身没有任何状态概念。例如，如果问一个后续问题：

In [26]:
messages_2=[HumanMessage(content='what is my name'),SystemMessage(content='answer word<10')]
chain.invoke(messages_2)


"I'm sorry, I don't know your name. Could you please tell me?"

### 为了绕过这个问题，我们需要将整个对话历史
### 传递给模型。让我们看看这样做会发生什么：

In [42]:
chain.invoke(
    [
        HumanMessage(content="Hi! I'm jack_five"),
        AIMessage(content="HI !!!"),
        HumanMessage(content="What's my name?"),
    ]
)

'Your name is jack_five.'

### 现在得到了好的回应

# 消息历史

#### 我们可以使用消息历史类来包装我们的模型，使其具有状态。 这将跟踪模型的输入和输出，并将其存储在某个数据存储中。 未来的交互将加载这些消息，并将其作为输入的一部分传递给链。

In [48]:
#我们可以使用消息历史类来包装我们的模型，使其具有状态。 这将跟踪模型的输入和输出，
#并将其存储在某个数据存储中。 未来的交互将加载这些消息，并将其作为输入的一部分传递给链。

#之后，我们可以导入相关类并设置我们的链，该链包装模型并添加此消息历史。这里的一个关键部分是我们作为 get_session_history 传入的函数。这个函数预计接受一个 session_id 并返回一个消息历史对象。这个 session_id 用于区分不同的对话，并应作为配置的
#一部分在调用新链时传入（我们将展示如何做到这一点）。

In [49]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

In [51]:
store = {}#用于存储会话 ID 和对应的 BaseChatMessageHistory 对象之间的映射。每个 session_id 都对应一个 BaseChatMessageHistory 实例。


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
# session_id: str：函数接收一个字符串参数 session_id，表示会话的唯一标识符。
# -> BaseChatMessageHistory：函数的返回类型是 BaseChatMessageHistory，表示该函数将返回一个 
# BaseChatMessageHistory 类型的对象。



In [52]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)
#新的模型，拥有了对话历史函数

#### 我们现在需要创建一个 config，每次都传递给可运行的部分。这个配置包含的信息并不是直接作为输入的一部分，但仍然是有用的。在这种情况下，我们想要包含一个 session_id。这应该看起来像：

In [60]:
config = {"configurable": {"session_id": "abc2"}}

In [61]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)


In [62]:
response.content

'Hello Bob! How can I assist you today?'

In [63]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
response.content

'Your name is Bob.'

#### 我们的聊天机器人现在记住了关于我们的事情。如果我们更改配置以引用不同的 session_id，我们可以看到它开始新的对话。

In [65]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, as an AI language model, I don't have access to your personal information. Please introduce yourself."

#### 这就是我们如何支持聊天机器人与多个用户进行对话的方式！现在，我们所做的只是为模型添加了一个简单的持久化层。我们可以通过添加提示词模板来使其变得更加复杂和个性化。

#### 提示词模板帮助将原始用户信息转换为大型语言模型可以处理的格式。在这种情况下，原始用户输入只是一个消息，我们将其传递给大型语言模型。现在让我们使其变得更复杂一些。首先，让我们添加一个带有一些自定义指令的系统消息（但仍然将消息作为输入）。接下来，我们将添加除了消息之外的更多输入。首先，让我们添加一个系统消息。为此，我们将创建一个 ChatPromptTemplate。我们将利用 MessagesPlaceholder 来传递所有消息。

In [71]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

#### 注意，这稍微改变了输入类型 - 我们现在传递的是一个包含 messages 键的字典，其中包含一系列消息，而不是传递消息列表。

In [73]:
response=chain.invoke({"messages":[HumanMessage(content='hi!i am bob')]})

In [75]:
response.content

'Hello Bob! How can I assist you today?'

### 我们现在可以将其包装在与之前相同的消息历史对象中

In [77]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [82]:
config = {"configurable": {"session_id": "abc4"}}

In [83]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

'Hello Jim! How can I assist you today?'

In [91]:
### 从下面的输入可以看到，在MessagesPlaceholder(variable_name="messages")
#的位置输入多个HumanMessage，虽然先输入的HumanMessage没有应答，但也保存在了
#store中 可以用于后续的回答

In [89]:
response = with_message_history.invoke(
    [HumanMessage(content="my brother'name is wang"),HumanMessage(content="introduce me a fiction book")],
    config=config,
)

response.content

'If you\'re looking for a great work of fiction, I would highly recommend "The Great Gatsby" by F. Scott Fitzgerald. The novel is set in the roaring 1920s and follows the wealthy and mysterious Jay Gatsby as he tries to win back his lost love, Daisy Buchanan. The book is a classic and is known for its beautiful prose, vivid imagery, and exploration of themes such as love, wealth, and the American Dream.'

In [90]:
response = with_message_history.invoke(
    [HumanMessage(content="what is my brother'name?")],
    config=config,
)

response.content

"Your brother's name is Wang."

### 现在让我们使我们的提示变得更复杂一点。假设提示模板现在看起来像这样：

In [93]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [94]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola Bob! ¿En qué puedo ayudarte hoy?'

### 现在让我们将这个更复杂的链封装在一个消息历史类中。这次，由于输入中有多个键，我们需要指定正确的键来保存聊天历史。

In [104]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [97]:
config = {"configurable": {"session_id": "abc11"}}

In [105]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'Hola Todd, ¿cómo puedo ayudarte hoy?'

# 编写自定义令牌计数器

### 编写一个自定义令牌计数器函数，该函数接受消息列表并返回一个整数。

In [116]:
from typing import List

# pip install tiktoken
import tiktoken
from langchain_core.messages import BaseMessage, ToolMessage


def str_token_counter(text: str) -> int:
    enc = tiktoken.get_encoding("o200k_base")
    return len(enc.encode(text))


def tiktoken_counter(messages: List[BaseMessage]) -> int:
    """Approximately reproduce https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb

    For simplicity only supports str Message.contents.
    """
    num_tokens = 3  # every reply is primed with <|start|>assistant<|message|>
    tokens_per_message = 3
    tokens_per_name = 1
    for msg in messages:
        if isinstance(msg, HumanMessage):
            role = "user"
        elif isinstance(msg, AIMessage):
            role = "assistant"
        elif isinstance(msg, ToolMessage):
            role = "tool"
        elif isinstance(msg, SystemMessage):
            role = "system"
        else:
            raise ValueError(f"Unsupported messages type {msg.__class__}")
        num_tokens += (
            tokens_per_message
            + str_token_counter(role)
            + str_token_counter(msg.content)
        )
        if msg.name:
            num_tokens += tokens_per_name + str_token_counter(msg.name)
    return num_tokens


trim_messages(
    messages,
    max_tokens=45,
    strategy="last",
    token_counter=tiktoken_counter,
)

[HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

## 管理对话历史

#### 构建聊天机器人时，一个重要的概念是如何管理对话历史。如果不加以管理，消息列表将无限增长，并可能溢出大型语言模型的上下文窗口。因此，添加一个限制您传入消息大小的步骤是很重要的。

#### 我们可以通过在提示前添加一个简单的步骤，适当地修改 messages 键，然后将该新链封装在消息历史类中来实现。
#### LangChain 提供了一些内置的助手来 管理消息列表。在这种情况下，我们将使用 trim_messages 助手来减少我们发送给模型的消息数量。修剪器允许我们指定希望保留的令牌数量，以及其他参数，例如是否希望始终保留系统消息以及是否允许部分消息：

In [123]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=60,    
    strategy="last",                #last or first   last是从后向前保留消息，会丢较前的输入
    token_counter=tiktoken_counter, #令牌计数器
    include_system=True,            #保留系统消息
    allow_partial=False,            #指定是否允许部分修剪。如果设置为 True，即使无法完全满足 max_tokens 的要求，也会返回尽可能多的消息；如果设置为 F
                                    #alse，则只有在能够完全满足 max_tokens 的情况下才会返回消息，否则返回空列表。
    start_on="human",               #指定从哪种类型的消息开始修剪
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

### 可以看到随着我们对修剪器的参数如max_tokens和strategy进行调整，输出结果将发生变化